In [116]:
import baltic as bt
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from datetime import datetime as dt
from datetime import timedelta
import time
#import pymc3
import math
import arviz as az
#from hpd import hpd
import scipy.stats as stats
from io import StringIO
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()
import seaborn as sns

from zipfile import ZipFile
import scipy as sp


import sys, subprocess, glob, os, shutil, re, importlib
from subprocess import call
import imp


%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patheffects as path_effects
import matplotlib.lines as mlines
from matplotlib.font_manager import FontProperties
import matplotlib.colors as clr
from matplotlib import rc
import textwrap as textwrap
from textwrap import wrap

import warnings
warnings.filterwarnings('ignore')

from scipy.special import binom

In [117]:
def read_in_Ne_changes_mascot(log_file_path):
    
    Ne_skyline_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                #print(line)
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    Ne_column_indices = []   # list to store column indices
                    Nes_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "Ne." in col:
                            Ne_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in Ne_column_indices:
                        name = line.split("\t")[n]
                        #deme = name.split(".")[1]# the syntax here is "Ne_region.1" where region is deme and 1 is interval 1
                        interval = name.split(".")[1]
                       
                        Nes_key[n] = name
                        Ne_skyline_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    Ne_skyline_dict["sample"].append(sample)

                    for index in Ne_column_indices:
                        name = Nes_key[index]
                        Ne_skyline_dict[name].append(line.split("\t")[index])
                    
                
    return(Ne_skyline_dict)

In [118]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "Ne." in i:
            #deme = i.split(".")[1]
           # print(deme)
            #interval = 
            #print(interval)
#             if "\n" in i.split(".")[2]:
#                 interval = i.split(".")[2][0:2]
#             else:
            interval = i.split(".")[1]
           # print(interval)
            #print(interval)
            #print(i)
            #next_interval = int(interval)+1
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_log = local_series.mean()
            median_log = np.median(local_series)
            mean_linear = math.exp(mean_log)
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            lower_hpd_linear_95 = math.exp(lower_hpd_log_95)
            upper_hpd_log_95 = hpd_95[1]
            upper_hpd_linear_95 = math.exp(upper_hpd_log_95)
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            lower_hpd_linear_50 = math.exp(lower_hpd_log_50)
            upper_hpd_log_50 = hpd_50[1]
            upper_hpd_linear_50 = math.exp(upper_hpd_log_50)
            
#             try:
#                 next_local_series = input_df["SkylineNe"+"."+ str(deme) +"." + str(next_interval)].astype('float').to_numpy()
#                 diff_series = np.subtract(local_series, next_local_series)
#                 #print(local_series)
#                 #print(next_local_series)
#                 #print(diff_series)
#                 diff_mean_log = diff_series.mean()
#                 diff_median_log = np.median(diff_series)
#                 diff_hpd_95 = az.hdi(diff_series, 0.95)
#                 diff_lower_hpd_log_95 = diff_hpd_95[0]
#                 diff_lower_hpd_linear_95 = math.exp(diff_lower_hpd_log_95)
#                 diff_upper_hpd_log_95 = diff_hpd_95[1]
#                 diff_upper_hpd_linear_95 = math.exp(diff_upper_hpd_log_95)
#                 diff_hpd_50 = az.hdi(diff_series, 0.50)
#                 diff_lower_hpd_log_50 = diff_hpd_50[0]
#                 diff_lower_hpd_linear_50 = math.exp(diff_lower_hpd_log_50)
#                 diff_upper_hpd_log_50 = diff_hpd_50[1]
#                 diff_upper_hpd_linear_50 = math.exp(diff_upper_hpd_log_50)
#             except KeyError:
#                 pass   
            
            try:
                local_df = pd.DataFrame.from_dict({"interval":interval, "mean_Ne_log":mean_log,"mean_Ne_linear":mean_linear, 
                                                   "median_Ne_log" : median_log, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   "upper_hpd_linear":upper_hpd_linear_95,"lower_hpd_linear":lower_hpd_linear_95,
                                                   "upper_hpd_linear_50":upper_hpd_linear_50, "lower_hpd_linear_50":lower_hpd_linear_50,
                                                  })
                new_df = new_df.append(local_df)
                #print(new_df)
            except:
                pass
            
    return(new_df)

In [119]:
#calculating transmission rate
def generate_summary_diff_df(input_df):
    
    
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "Ne." in i:
            #deme = i.split(".")[1]
           # print(deme)
            #interval = 
            #print(interval)
#             if "\n" in i.split(".")[2]:
#                 interval = i.split(".")[2][0:2]
#             else:
            interval = i.split(".")[1]
            next_interval = int(interval)+6 #averaging over two weeks to reduce noise
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
           
            try:
                new_df["Ne"+ ".diff." + str(interval)] = (52/6)*(np.log(input_df[i].astype("float")) - np.log(input_df["Ne"+"." + str(next_interval)].astype('float')))
            
            
            except KeyError:
                pass 
            
            
    return(new_df)

In [120]:
log_file_path = "../multitree_coalescent/results/multicoal_updated_smoothed_multicoal_updated_case_prior_la_clusters_with_metadata_11_07.log"


In [121]:
Ne_skyline = read_in_Ne_changes_mascot(log_file_path)

#burnin
Ne_df = pd.DataFrame.from_dict(Ne_skyline)
print(len(Ne_df))
Ne_df

burnin_percent = 0.3
print(len(Ne_df))
rows_to_remove = int(len(Ne_df)* burnin_percent)
Ne_df = Ne_df.iloc[rows_to_remove:]

print(len(Ne_df))
Ne_df = Ne_df.reset_index()
Ne_df.head()

12127
12127
8489


,index,sample,Ne.1,Ne.2,Ne.3,Ne.4,Ne.5,Ne.6,Ne.7,Ne.8,...,Ne.121,Ne.122,Ne.123,Ne.124,Ne.125,Ne.126,Ne.127,Ne.128,Ne.129,Ne.130
0,3638,181900000,-1.0745922835048785,-0.874702863586052,-0.6351692499305073,-1.0587503758396677,-0.19914675211154817,0.2858900357644437,2.6117269008838355,2.9503084522929615,...,4.855472162850968,3.0060740438757376,2.2337246669266615,-0.4494613304119273,0.6582758904769195,-0.5187818904114999,0.21298452806734378,-0.5436558300531796,-0.8508816371323054,-0.44351409994991675
1,3639,181950000,-3.1642580778262093,-1.410494413579376,-1.133129212546593,-0.5344767764995205,-0.23577290057448408,0.8514913083078179,0.9781500052544273,0.15363204002764,...,0.2861408159633883,0.58007636459387,-0.46154649207481074,-1.3207477444823916,-1.1791021419829126,-1.7590486795405182,-0.09779235930785816,-1.2528677148891991,1.326696135182568,2.84775544637355
2,3640,182000000,-0.8674195095775974,-0.9126206808119993,0.3298424773750454,1.686745667296047,1.1252427898904045,1.6692564647058006,2.4173367057519513,0.9595780784805373,...,4.1652328322440235,3.2902953360511464,2.3155767413011725,-0.6423124300511176,-0.17198396662254944,0.07804527830476615,1.212648420063384,0.7933170479880508,-0.9456900055328017,-2.798897667834083
3,3641,182050000,-5.222123169337881,-1.737388380947547,-0.9249751538934652,-0.6891118139333139,0.9832054375132788,0.12972714517329073,0.6388428047090468,0.39487780638640974,...,1.947589618996802,-0.6340937127042654,-0.9113388631766622,-0.616033493781511,-2.4116394242590484,-2.576222782138571,-1.8893431148368296,-1.0346560864883834,-0.04453559905488881,0.8269354416705376
4,3642,182100000,-0.5640622566429523,-0.5312603661630072,-0.5806635203581161,0.22135300297673605,0.7054233717303316,2.4424620632962557,1.314888390549833,2.6520588575903914,...,5.955953965773119,3.699816165608087,1.9931979560819253,0.5900036703598904,-0.10019050134108497,-0.011433049687306721,-0.2483850378502726,-1.0568843920422906,-1.545659174515696,-2.655407450531089


In [122]:
exp_ne_df = Ne_df.copy()
# Select only numeric columns excluding the first two columns
numeric_columns = exp_ne_df.columns[2:]  # Exclude the first two columns

# Apply the exponential function only to numeric columns
exp_ne_df[numeric_columns] = exp_ne_df[numeric_columns].apply(pd.to_numeric, errors='coerce').apply(np.exp)


uninfectious_rate = 365/4.5

#taken from https://www.medrxiv.org/content/10.1101/2022.08.17.22278897v1.full.pdf
incubation_period = 365/8

#calculate transmission rate based on SEIR dynamics
ne_diff_summary = generate_summary_diff_df(exp_ne_df)
seir_growth_rate = ((ne_diff_summary*2 + uninfectious_rate + incubation_period)**2 - (incubation_period- uninfectious_rate)**2)/(4*incubation_period)
seir_growth_rate.head()

,Ne.diff.1,Ne.diff.2,Ne.diff.3,Ne.diff.4,Ne.diff.5,Ne.diff.6,Ne.diff.7,Ne.diff.8,Ne.diff.9,Ne.diff.10,...,Ne.diff.115,Ne.diff.116,Ne.diff.117,Ne.diff.118,Ne.diff.119,Ne.diff.120,Ne.diff.121,Ne.diff.122,Ne.diff.123,Ne.diff.124
0,14.737482,13.113620,38.086596,51.880198,103.915440,88.025461,121.110142,83.724014,96.259850,55.231898,...,38.856093,76.123312,71.194495,177.872976,116.863926,248.002951,228.356272,187.311543,171.034084,80.967995
1,9.635732,47.483803,52.000933,95.066416,79.978836,90.302978,84.417608,69.558120,57.093146,44.725678,...,139.219482,137.228117,137.630278,202.422494,192.307618,198.341160,90.596615,130.768493,43.325290,9.364550
2,19.796290,41.810050,82.460923,152.504093,142.250141,139.848999,82.193925,40.347168,25.466259,23.989452,...,50.667330,87.852906,66.494106,175.619647,151.730959,231.671083,166.543633,151.487887,177.132597,140.685479
3,-3.435293,37.263645,62.824828,65.133163,111.668372,104.490500,106.587714,63.632416,42.207697,63.398598,...,103.172583,157.927945,168.702191,144.909754,250.992689,198.416842,197.718202,91.018424,61.480547,49.800768
4,41.689200,21.158170,61.981693,68.566195,110.133574,164.836721,111.372418,118.827963,48.918349,81.441164,...,22.209245,57.026307,73.033300,117.532853,155.167554,189.137336,293.846089,232.873128,186.922910,176.581046


In [123]:
ne_summary = generate_summary_df(Ne_df)
test = ne_summary
test['days'] = (test.interval.astype(int) -3 )*7
test['date'] = dt.strptime("2024-10-06",  "%Y-%m-%d") - test.days.map(timedelta)
test = test.reset_index()

columns = ["mean_Ne_linear", "upper_hpd_linear_50","lower_hpd_linear_50", "lower_hpd_linear", "upper_hpd_linear"]
for column in columns:
    # Create a new column for the moving average
    test[f'{column}_MA'] = test[column].rolling(3, min_periods =1).mean()



In [124]:
#domain = ['CentralEurope', 'NorthAmerica', 'SouthAmerica' ,"SouthernEurope", "WesternEurope"]
range_ = ['#EEC060', '#2664A5', '#A76BB1', "#EEA160", "#356D4C"]
line_ne = alt.Chart(test).mark_area(interpolate='monotone', opacity = 1, clip = True, color = "#7c5981").encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False,tickCount = "month",  format="%B %Y")),
    alt.Y('lower_hpd_linear_50_MA',axis=alt.Axis(title="Effective Population Size", grid=False),scale=alt.Scale(domain=(0, 60))),
    alt.Y2('upper_hpd_linear_50_MA' ),
    
).properties(
    width=850,
    height=300)
# ).transform_filter(
#      (datum.upper_hpd_linear_50_MA < 50) 
#  )
band_ne = alt.Chart(test).mark_area(interpolate='monotone', opacity = 0.5, clip = True, color = "#7c5981").encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False, format="%B %Y")),
    alt.Y('lower_hpd_linear_MA',axis=alt.Axis(title="Effective Population Size", grid=False)),#,scale=alt.Scale(domain=(0, 13))),
    alt.Y2('upper_hpd_linear_MA' ),
    
).properties(
    width=850,
    height=300
)

line_ne  + band_ne

# band = alt.Chart(test).mark_area(
#     opacity=0.3, interpolate='monotone'
# ).encode(
#     alt.X('date:T', axis=alt.Axis(title="", grid=False)),
#     alt.Y('lower_hpd_linear'),#axis=None),#, scale=alt.Scale(domain=(0, 13))),
#     alt.Y2('upper_hpd_linear'),
#     color=alt.Color('deme:N', title = "Region", legend=alt.Legend(orient = "right",offset = -135, labelFontSize = 16, titleFontSize = 16, symbolSize = 110))
# ).properties(
#     width=850,
#     height=400
# ).transform_filter(
#     (datum.upper_hpd_linear < 20) &(datum.upper_hpd_linear_50 < 30)
# )
# band
# ne_plot = line + band
# (ne_plot).configure_axis(
#     labelFontSize=20,
#     titleFontSize=20
# )


alt.LayerChart(...)

In [125]:
la_mpox_cases_df = pd.read_csv("../mpox_build/cluster_assignment/scripts/monkeypox_data.csv")
la_mpox_cases_df = la_mpox_cases_df.dropna(how = "all").dropna(axis = "columns", how = "all")
la_mpox_cases_df = la_mpox_cases_df.rename(columns= {"Unnamed: 0": "date"}); la_mpox_cases_df.head()

,date,cases
0,2022-05-19,1
1,2022-05-20,0
2,2022-05-21,0
3,2022-05-22,0
4,2022-05-23,0


In [126]:
cases = alt.Chart(la_mpox_cases_df).mark_bar(interpolate='monotone', opacity = 0.5, size = 1).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False, format="%B %Y")),
    alt.Y('cases',axis=alt.Axis(title="cases", grid=False))
).properties(
    width=850,
    height=300
)

(cases + line_ne).resolve_scale(y = "independent")

alt.LayerChart(...)

In [127]:
#switching to migration

In [128]:
def read_in_forward_migration_rates_mascot(log_file_path):
    
    mig_rates_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            #print(line_number)
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    mig_column_indices = []   # list to store column indices
                    mig_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "immigrationRate" in col:
                            mig_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in mig_column_indices:
                        name = line.split("\t")[n]
                        interval = name.split(".")[1]# the syntax here is "NeLog.state01" where 0 is deme and 1 is interval 1
                        #interval = name.split(".")[2]
                       
                        mig_key[n] = name
                        mig_rates_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    mig_rates_dict["sample"].append(sample)

                    for index in mig_column_indices:
                        name = mig_key[index]
                        mig_rates_dict[name].append(line.split("\t")[index])
                    
                
                
                
    return(mig_rates_dict)

In [129]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "immigrationRate." in i:
            interval = i.split(".")[1]
           # print(interval)
            #print(interval)
            #print(i)
            #next_interval = int(interval)+1
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_log = local_series.mean()
            median_log = np.median(local_series)
            mean_linear = math.exp(mean_log)
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            lower_hpd_linear_95 = math.exp(lower_hpd_log_95)
            upper_hpd_log_95 = hpd_95[1]
            upper_hpd_linear_95 = math.exp(upper_hpd_log_95)
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            lower_hpd_linear_50 = math.exp(lower_hpd_log_50)
            upper_hpd_log_50 = hpd_50[1]
            upper_hpd_linear_50 = math.exp(upper_hpd_log_50)
            
#             try:
#                 next_local_series = input_df["SkylineNe"+"."+ str(deme) +"." + str(next_interval)].astype('float').to_numpy()
#                 diff_series = np.subtract(local_series, next_local_series)
#                 #print(local_series)
#                 #print(next_local_series)
#                 #print(diff_series)
#                 diff_mean_log = diff_series.mean()
#                 diff_median_log = np.median(diff_series)
#                 diff_hpd_95 = az.hdi(diff_series, 0.95)
#                 diff_lower_hpd_log_95 = diff_hpd_95[0]
#                 diff_lower_hpd_linear_95 = math.exp(diff_lower_hpd_log_95)
#                 diff_upper_hpd_log_95 = diff_hpd_95[1]
#                 diff_upper_hpd_linear_95 = math.exp(diff_upper_hpd_log_95)
#                 diff_hpd_50 = az.hdi(diff_series, 0.50)
#                 diff_lower_hpd_log_50 = diff_hpd_50[0]
#                 diff_lower_hpd_linear_50 = math.exp(diff_lower_hpd_log_50)
#                 diff_upper_hpd_log_50 = diff_hpd_50[1]
#                 diff_upper_hpd_linear_50 = math.exp(diff_upper_hpd_log_50)
#             except KeyError:
#                 pass   
            
            try:
                local_df = pd.DataFrame.from_dict({"interval":interval, "mean_mig_log":mean_log,"mean_mig_linear":mean_linear, 
                                                   "median_mig_log" : median_log, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   "upper_hpd_linear":upper_hpd_linear_95,"lower_hpd_linear":lower_hpd_linear_95,
                                                   "upper_hpd_linear_50":upper_hpd_linear_50, "lower_hpd_linear_50":lower_hpd_linear_50,
                                                  })
                new_df = new_df.append(local_df)
                #print(new_df)
            except:
                pass
            
   
            

            
    return(new_df)

In [130]:
migration_rates_f = read_in_forward_migration_rates_mascot(log_file_path)
mig_df_f = pd.DataFrame.from_dict(migration_rates_f)

#remove burnin
burnin_percent = 0.3
print(len(mig_df_f))
rows_to_remove = int(len(mig_df_f)* burnin_percent)
mig_df_f = mig_df_f.iloc[rows_to_remove:]

print(len(mig_df_f))
mig_df_f = mig_df_f.reset_index()
mig_df_f.head()

12127
8489


,index,sample,immigrationRate.1,immigrationRate.2,immigrationRate.3,immigrationRate.4,immigrationRate.5,immigrationRate.6,immigrationRate.7,immigrationRate.8,...,immigrationRate.120,immigrationRate.121,immigrationRate.122,immigrationRate.123,immigrationRate.124,immigrationRate.125,immigrationRate.126,immigrationRate.127,immigrationRate.128,immigrationRate.129
0,3638,181900000,-1.5855324600606988,-2.322150850849798,-2.729530356452055,-2.1853453868504724,-2.580805619913272,-1.6506311555008304,-2.2501586385699284,-3.1887142344897668,...,2.41625667254899,2.369593127742671,1.5833564662200208,2.0803237561730907,3.510187095466377,3.199981155255794,4.025963890370722,4.226170509484097,4.109889978700479,3.8738643859518893
1,3639,181950000,-0.6722157743101338,-0.4198411115919247,-1.0025328090481784,-1.271477851266802,-1.2427955837295697,-1.834602035709103,-1.9671380182646474,-1.2772619743099378,...,2.2123229179399186,3.1209596046928105,3.4822097277187516,3.293496772425086,3.240972357422667,4.025638951204985,4.2893667876222,4.375337398990272,5.632471961192109,6.865937766967239
2,3640,182000000,0.09933443987237994,-0.01604051087308439,-1.9473684244994418,-1.6329971656658044,-2.3557800284725814,-2.020638106854897,-1.613232743552353,-1.0903965833047682,...,1.6033536998254285,2.0286066263755997,2.444152851621375,2.588923249377911,2.98461491426509,3.2219022180193204,3.8543662191146355,3.32730125415223,4.226379877013783,3.651303248299946
3,3641,182050000,-0.5715722402824535,-0.4875204856544497,-0.6506984794558734,-0.48696331646493896,-0.5959730337213309,-0.7832394039585334,-0.888469504476093,-1.4439966964201614,...,2.8775893710816094,3.5501941774317074,3.443770749977927,4.220542101179485,5.098670603581778,4.97056477564744,5.631834056692215,5.150311164714823,5.251257087039374,5.079251831764074
4,3642,182100000,-3.5082068996386857,-2.703410192058877,-2.6637853864400496,-2.2040823458726178,-2.341025932090607,-2.373257724817102,-2.108919381466737,-1.4955870010253116,...,1.647711903261984,2.466863851247571,2.015723858214982,1.98909977175114,3.2211836291507026,3.3121934052386504,3.5526494205445758,3.8280798830923115,3.442620249553967,1.947173898866242


In [131]:
mig_summary = generate_summary_df(mig_df_f)
mig_summary.head()

,interval,mean_mig_log,mean_mig_linear,median_mig_log,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,upper_hpd_linear,lower_hpd_linear,upper_hpd_linear_50,lower_hpd_linear_50
0,1,-1.598947,0.202109,-1.445521,1.413787,-4.797268,-0.008162,-2.072094,4.111497,0.008252,0.991872,0.125922
0,2,-1.604042,0.201082,-1.427215,1.178321,-4.682613,-0.260987,-2.199592,3.248916,0.009255,0.770291,0.110848
0,3,-1.645098,0.192994,-1.455310,0.875015,-4.603419,-0.372315,-2.149723,2.398912,0.010018,0.689137,0.116516
0,4,-1.688555,0.184786,-1.517993,0.806292,-4.368627,-0.467751,-2.116889,2.239588,0.012669,0.626409,0.120406
0,5,-1.719082,0.179231,-1.549641,0.476439,-4.440835,-0.466174,-2.046646,1.610331,0.011786,0.627398,0.129167


In [132]:
#format dates
test_mig = mig_summary
test_mig['days'] = (test_mig.interval.astype(int) -3 )*7
test_mig['date'] = dt.strptime("2024-10-06",  "%Y-%m-%d") - test_mig.days.map(timedelta)

In [133]:
#domain = ['CentralEurope', 'NorthAmerica', 'SouthAmerica' ,"SouthernEurope", "WesternEurope"]
range_ = ['#EEC060', '#2664A5', '#A76BB1', "#EEA160", "#356D4C"]
line_mig = alt.Chart(test_mig).mark_area(interpolate='monotone', opacity = 0.8).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False,tickCount = "month",  format="%B %Y")),
    alt.Y('lower_hpd_linear_50',axis=alt.Axis(title="Immigration Rate (migration events/lineage/year)", grid=False)),#,scale=alt.Scale(domain=(0, 13))),
    alt.Y2('upper_hpd_linear_50' ),
    
).properties(
    width=850,
    height=300
)

band_mig = alt.Chart(test_mig).mark_area(interpolate='monotone', opacity = 0.2, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False,tickCount = "month",  format="%B %Y")),
    alt.Y('lower_hpd_linear',axis=alt.Axis(title="Immigration Rate (migration events/lineage/year)", grid=False), scale=alt.Scale(domain=(0, 100))),
    alt.Y2('upper_hpd_linear' ),
    
).properties(
    width=850,
    height=300
)


mig_plot = (line_mig + band_mig)
mig_plot

alt.LayerChart(...)

In [134]:
## percentage of cases due to introductions

In [135]:
def generate_percent_intro_df(input_df):
    
    temp_df = pd.DataFrame()
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "immigrationRate" in i:
        
            interval = i.split(".")[1]
            #deme = i.split(".")[2]

            try:
                temp_df["total."+ str(interval)] = seir_growth_rate["Ne.diff." + str(interval)].astype("float") +  input_df[i].astype("float").apply(np.exp)
                new_df["intro.percent"+ "." + str(interval)] = input_df[i].astype("float").apply(np.exp).div(temp_df["total."+ str(interval)], axis = 0) 

            except KeyError: #this was added because not all regions have equal time periods for their epidemics and it was throwing an error everytime it had to switch deme
                pass 
                  
    return(new_df)

In [136]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "percent" in i:
            interval = i.split(".")[2]
            #interval = i.split(".")[3]
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_percent = local_series.mean()
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            upper_hpd_log_95 = hpd_95[1]
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            upper_hpd_log_50 = hpd_50[1]
            

            
            
            try:
                local_df = pd.DataFrame.from_dict({ "interval":interval, "mean_percent":mean_percent, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                new_df = new_df.append(local_df)
            except:
                pass
            
    return(new_df)


In [137]:
percent_df = generate_percent_intro_df(mig_df_f)
percent_df.head()

,intro.percent.1,intro.percent.2,intro.percent.3,intro.percent.4,intro.percent.5,intro.percent.6,intro.percent.7,intro.percent.8,intro.percent.9,intro.percent.10,...,intro.percent.115,intro.percent.116,intro.percent.117,intro.percent.118,intro.percent.119,intro.percent.120,intro.percent.121,intro.percent.122,intro.percent.123,intro.percent.124
0,0.013709,0.007422,0.001710,0.002163,0.000728,0.002176,0.000869,0.000492,0.000532,0.004605,...,0.066241,0.025461,0.024054,0.011522,0.041494,0.043224,0.044732,0.025347,0.044722,0.292377
1,0.050321,0.013651,0.007007,0.002941,0.003595,0.001765,0.001654,0.003992,0.009596,0.011678,...,0.037116,0.027874,0.052716,0.040457,0.054513,0.044038,0.200134,0.199213,0.383377,0.731852
2,0.052842,0.022996,0.001727,0.001279,0.000666,0.000947,0.002418,0.008261,0.021041,0.010384,...,0.034033,0.021929,0.039245,0.013549,0.020893,0.021001,0.043661,0.070676,0.069916,0.123260
3,-0.196693,0.016214,0.008235,0.009346,0.004910,0.004354,0.003844,0.003695,0.013588,0.023774,...,0.021872,0.012688,0.027579,0.057776,0.037322,0.082203,0.149739,0.255919,0.525433,0.766856
4,0.000718,0.003156,0.001123,0.001607,0.000873,0.000565,0.001089,0.001883,0.007551,0.003961,...,0.052033,0.037113,0.039519,0.031187,0.027642,0.026733,0.038561,0.031226,0.037630,0.124271


In [138]:
summary_percent_df = generate_summary_df(percent_df)
summary_percent_df = summary_percent_df.reset_index()

#making sure that any numbers >1 are excluded as outliers from stochastic noise
percent_df =pd.DataFrame(np.where(percent_df <1, percent_df, 1), columns=percent_df.columns )
percent_df =pd.DataFrame(np.where(percent_df >0, percent_df, 0), columns=percent_df.columns )

#format dates
summary_percent_df['days'] = (summary_percent_df.interval.astype(int)-3)*7 #the 1.5 adjustment is made due to the fact that we take the difference of Nes over three weeks 
summary_percent_df['date'] = dt.strptime("2024-10-06",  "%Y-%m-%d") - summary_percent_df.days.map(timedelta)

summary_percent_df.head()

,index,interval,mean_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,days,date
0,0,1,0.046796,0.243972,-8.429498e-02,0.008882,4.462761e-07,-14,2024-10-20
1,0,2,0.018382,0.076874,2.218892e-07,0.006611,2.218892e-07,-7,2024-10-13
2,0,3,0.009436,0.030768,6.240007e-08,0.004248,6.240007e-08,0,2024-10-06
3,0,4,0.004805,0.016271,1.453679e-07,0.002727,6.757033e-06,7,2024-09-29
4,0,5,0.002981,0.009792,2.780076e-07,0.001797,2.179152e-06,14,2024-09-22


In [139]:
columns = ["mean_percent", "upper_hpd_log_95","lower_hpd_log_95", "upper_hpd_log_50", "lower_hpd_log_50"]
for column in columns:
    # Create a new column for the moving average
    summary_percent_df[f'{column}_MA'] = summary_percent_df[column].rolling(4, min_periods =1).mean()


In [210]:
line1 = alt.Chart(summary_percent_df).mark_area(interpolate='monotone', opacity = 0.9, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month",grid=False,  format="%b %Y")),
    alt.Y('lower_hpd_log_50_MA',title = "Percent of new cases due to introductions", axis=alt.Axis( grid=False, format='%', ), scale=alt.Scale(domain=(0.0, .3))),
    alt.Y2('upper_hpd_log_50_MA' )
).properties(
    width=850,
    height=300
)

band = alt.Chart(summary_percent_df).mark_area(interpolate='monotone', opacity = 0.2, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month",grid=False,  format="%b %Y")),
    alt.Y('lower_hpd_log_95_MA',title = "Percent of new cases due to introductions", axis=alt.Axis( grid=False, format='%', ), scale=alt.Scale(domain=(0.0, .3))),
    alt.Y2('upper_hpd_log_95_MA' )
).properties(
    width=850,
    height=300
)


intro_plot = (line1 + band).resolve_scale(y= "shared") ; intro_plot

alt.LayerChart(...)

In [141]:
## calculate Rt

In [142]:
def generate_local_and_intro_rt(input_df):
    
    
    new_df = pd.DataFrame()
    incubation_period = 365/8
    uninfectious_rate = 365/4.5
    
    for i in input_df.columns.tolist():
        if "Ne" in i:
            interval = i.split(".")[2]
            new_df["rt" +  "." + str(interval)] = (1+ (input_df[i].astype("float") / uninfectious_rate)) * (1+ (input_df[i].astype("float") / incubation_period))
            

            
    return(new_df)

In [143]:
def generate_local_rt(input_df):
    
    
    new_df = pd.DataFrame()
    incubation_period = 365/8
    uninfectious_rate = 365/4.5
    
    for i in input_df.columns.tolist():
        if "Ne" in i:
            interval = i.split(".")[2]
            try:
                new_df["rt" + "." + str(interval)] = ((1+ (input_df[i].astype("float") / uninfectious_rate)) * (1+ (input_df[i].astype("float") / incubation_period))) *  (1- percent_df["intro.percent."+str(interval)].astype("float"))
                
            except KeyError:
                pass

            
    return(new_df)

In [144]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_rt_summary_df(input_df):
    
    
    new_df = pd.DataFrame()
    count = 0
    for i in input_df.columns.tolist():
        #print(i)
        if "rt" in i:
            if count %1 == 0:
                interval = i.split(".")[1]
                #interval = i.split(".")[2]
                local_series = input_df[i].astype('float').to_numpy()
                mean_percent = local_series.mean()
                median_percent = np.median(local_series)

                hpd_95 = az.hdi(local_series, 0.95)
                lower_hpd_log_95 = hpd_95[0]
                upper_hpd_log_95 = hpd_95[1]
                hpd_50 = az.hdi(local_series, 0.50)
                lower_hpd_log_50 = hpd_50[0]
                upper_hpd_log_50 = hpd_50[1]

                try:
                    local_df = pd.DataFrame.from_dict({"interval":interval, "mean_percent":mean_percent, "median_percent":median_percent,
                                                       "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                       "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                    new_df = new_df.append(local_df)
                except:
                    pass
            count+=1

    return(new_df)

In [145]:
# a line to show an Rt of 1
one_line = alt.Chart(pd.DataFrame({'y': [1.0]})).mark_rule(strokeDash=[1,1]).encode(y='y').properties(
    width=850,
    height=300
)

In [146]:
#read in data and calculate based on posterior set of trees
rt_local_and_intro_df = generate_local_and_intro_rt(ne_diff_summary)
rt_local_df = generate_local_rt(ne_diff_summary)

#summarize data
summary_rt_local_and_intro_df = generate_rt_summary_df(rt_local_and_intro_df)
summary_rt_local_df = generate_rt_summary_df(rt_local_df)

summary_rt_local_and_intro_df.head()

,interval,mean_percent,median_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50
0,1,0.436721,0.390449,1.057037,-0.085067,0.502165,0.080820
0,2,0.494751,0.467068,1.008969,0.009062,0.573409,0.243036
0,3,0.712384,0.697202,1.231638,0.235945,0.845749,0.505999
0,4,1.043971,1.018590,1.680226,0.433522,1.194128,0.790299
0,5,1.485343,1.443405,2.290086,0.774697,1.620915,1.127375


In [147]:
summary_rt_local_and_intro_df['days'] = (summary_rt_local_and_intro_df.interval.astype(int) -3) *7
summary_rt_local_and_intro_df['date'] = dt.strptime("2024-10-06",  "%Y-%m-%d") - summary_rt_local_and_intro_df.days.map(timedelta)

summary_rt_local_df['days'] = (summary_rt_local_df.interval.astype(int) -3) *7
summary_rt_local_df['date'] = dt.strptime("2024-10-06",  "%Y-%m-%d") - summary_rt_local_df.days.map(timedelta)

summary_rt_local_and_intro_df["contribution"] = "Local + Introductions"
summary_rt_local_df["contribution"] = "Local"

In [148]:
columns = ["mean_percent","median_percent", "upper_hpd_log_95","lower_hpd_log_95", "upper_hpd_log_50", "lower_hpd_log_50"]
for column in columns:
    # Create a new column for the moving average
    summary_rt_local_and_intro_df[f'{column}_MA'] = summary_rt_local_and_intro_df[column].rolling(6, min_periods =1).mean()
    summary_rt_local_df[f'{column}_MA'] = summary_rt_local_df[column].rolling(6, min_periods =1).mean()


In [149]:
combined_rt_df = pd.concat([summary_rt_local_and_intro_df,summary_rt_local_df], ignore_index = True )

In [150]:
rt_line2 = alt.Chart(combined_rt_df).mark_area(interpolate='monotone', opacity = 0.7, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month", grid=False,format="%B %Y")),
    alt.Y('lower_hpd_log_50_MA',axis=alt.Axis(title="Phylo-estimated Rt", grid=False)),
    alt.Y2('upper_hpd_log_50_MA' ),
    alt.Color("contribution")
).properties(
    width=850,
    height=300
)
rt_band2 = alt.Chart(combined_rt_df).mark_area(
    opacity=0.3, interpolate='monotone'
).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False,format="%B %Y")),
    alt.Y('lower_hpd_log_95_MA', axis=alt.Axis(title="Phylo-estimated Rt", grid=False)),# scale=alt.Scale(domain=(0.6, 1.3))),
    alt.Y2('upper_hpd_log_95_MA'), 
    alt.Color("contribution")
).properties(
    width=850,
    height=300
)

stream_plot_rt = rt_line2  + one_line

stream_plot_rt

alt.LayerChart(...)

In [214]:
line_simple_intro = alt.Chart(combined_rt_df).mark_line(interpolate='monotone', opacity = 1, clip = True, strokeDash=[5,1]).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month", grid=False,format="%B %Y")),
    alt.Y('mean_percent_MA',scale=alt.Scale(domain=[0.5, 2]),axis=alt.Axis(title="Phylo-estimated Rt", grid=False)),
    alt.Color("contribution", legend=alt.Legend(offset=-200))

).properties(
    width=850,
    height=300
)


line_rt = line_simple_intro+ one_line; line_rt

alt.LayerChart(...)

In [215]:
alt.layer(intro_plot, line_rt).resolve_scale(y= "independent")

alt.LayerChart(...)

In [153]:
alt.layer(mig_plot, line_rt).resolve_scale(y= "independent")

alt.LayerChart(...)

In [154]:
cases_ne = (cases + line_ne).resolve_scale(y= "shared")

In [155]:
alt.layer(cases_ne, line_rt).resolve_scale(y = "independent")

alt.LayerChart(...)

In [156]:
## compare phylo rt to microsim Rt

In [192]:
sim_df = pd.read_csv("../data/results_sep13_2024.csv")
sim_df["week"] = sim_df.date.str.rsplit(" ", 0).str[-0].tolist()
sim_df = sim_df.dropna()
sim_df = sim_df.replace("#DIV/0!", 0)

sim_df["Model (FoI=1) R_e"]= sim_df["Model (FoI=1) R_e"].astype("float")
sim_df["Model (FoI=1.25) R_e"]= sim_df["Model (FoI=1.25) R_e"].astype("float")
sim_df["Model (FoI=1.5) R_e"]= sim_df["Model (FoI=1.5) R_e"].astype("float")

sim_df.head()

,date,Empirical # of diagnosed cases,Model (FoI=1) diagnosed cases,Model (FoI=1) incidence,Model (FoI=1) symp,Model (FoI=1) R_e,Model (FoI=1.25) diagnosed cases,Model (FoI=1.25) incidence,Model (FoI=1.25) symp,Model (FoI=1.25) R_e,Model (FoI=1.5) diagnosed cases,Model (FoI=1.5) incidence,Model (FoI=1.5) symp,Model (FoI=1.5) R_e,Model (FoI=2) diagnosed cases,Model (FoI=2) incidence,week
1,7/10/22 - 7/16/22,125,123,140,279,2.320442,123,140,279,2.320442,123,140,279,2.320442,123,140,7/10/22
2,7/17/22 - 7/23/22,185,179,249,392,2.677419,179,249,392,2.677419,179,249,392,2.677419,179,249,7/17/22
3,7/24/22 - 7/30/22,221,202,302,495,2.311224,202,302,495,2.311224,202,302,495,2.311224,202,302,7/24/22
4,7/31/22 - 8/6/22,264,226,391,619,2.369697,226,391,619,2.369697,226,391,619,2.369697,226,391,7/31/22
5,8/7/22 - 8/13/22,261,258,143,645,0.693053,258,143,645,0.693053,258,143,645,0.693053,258,143,8/7/22


In [193]:
columns = ["Model (FoI=1) R_e","Model (FoI=1.25) R_e", "Model (FoI=1.5) R_e"]
for column in columns:
    # Create a new column for the moving average
    sim_df[f'{column}_MA'] = sim_df[column].rolling(3, min_periods =1, center = True).mean()



In [194]:
sim_df.head()

,date,Empirical # of diagnosed cases,Model (FoI=1) diagnosed cases,Model (FoI=1) incidence,Model (FoI=1) symp,Model (FoI=1) R_e,Model (FoI=1.25) diagnosed cases,Model (FoI=1.25) incidence,Model (FoI=1.25) symp,Model (FoI=1.25) R_e,Model (FoI=1.5) diagnosed cases,Model (FoI=1.5) incidence,Model (FoI=1.5) symp,Model (FoI=1.5) R_e,Model (FoI=2) diagnosed cases,Model (FoI=2) incidence,week,Model (FoI=1) R_e_MA,Model (FoI=1.25) R_e_MA,Model (FoI=1.5) R_e_MA
1,7/10/22 - 7/16/22,125,123,140,279,2.320442,123,140,279,2.320442,123,140,279,2.320442,123,140,7/10/22,2.498931,2.498931,2.498931
2,7/17/22 - 7/23/22,185,179,249,392,2.677419,179,249,392,2.677419,179,249,392,2.677419,179,249,7/17/22,2.436362,2.436362,2.436362
3,7/24/22 - 7/30/22,221,202,302,495,2.311224,202,302,495,2.311224,202,302,495,2.311224,202,302,7/24/22,2.452780,2.452780,2.452780
4,7/31/22 - 8/6/22,264,226,391,619,2.369697,226,391,619,2.369697,226,391,619,2.369697,226,391,7/31/22,1.791325,1.791325,1.791325
5,8/7/22 - 8/13/22,261,258,143,645,0.693053,258,143,645,0.693053,258,143,645,0.693053,258,143,8/7/22,1.245723,1.245723,1.245723


In [205]:
long_sims_df = pd.melt(sim_df, id_vars='week', value_vars=["Empirical # of diagnosed cases_MA",'Model (FoI=1) R_e_MA', 'Model (FoI=1.25) R_e_MA', 'Model (FoI=1.5) R_e_MA'])
long_sims_df.week = pd.to_datetime(long_sims_df.week)



KeyError: "The following 'value_vars' are not present in the DataFrame: ['Empirical # of diagnosed cases_MA']"

In [206]:
subset_sims_df = long_sims_df[long_sims_df.week >"2023-03-01"]
subset_sims_df.variable[subset_sims_df.variable == "Empirical # of diagnosed cases_MA"] = "Empirical"
subset_sims_df.variable[subset_sims_df.variable == "Model (FoI=1) R_e_MA"] = "Model (FoI = 1)"
subset_sims_df.variable[subset_sims_df.variable == "Model (FoI=1.25) R_e_MA"] = "Model (FoI = 1.25)"
subset_sims_df.variable[subset_sims_df.variable == "Model (FoI=1.5) R_e_MA"] = "Model (FoI = 1.5)"



In [207]:
subset_sims_df

,week,variable,value
34,2023-03-05,Empirical # of diagnosed cases,1.000000
35,2023-03-12,Empirical # of diagnosed cases,0.000000
36,2023-03-19,Empirical # of diagnosed cases,0.000000
37,2023-03-26,Empirical # of diagnosed cases,0.000000
38,2023-04-02,Empirical # of diagnosed cases,0.000000
...,...,...,...
439,2024-07-21,Model (FoI = 1.5),1.022222
440,2024-07-28,Model (FoI = 1.5),1.293651
441,2024-08-04,Model (FoI = 1.5),1.365079
442,2024-08-11,Model (FoI = 1.5),1.920635


In [208]:
cases_bar =  alt.Chart(subset_sims_df, title = "").mark_bar(interpolate='monotone', opacity = 0.8 ,color = "#f58518", clip = True).encode(
    alt.X('week', axis=alt.Axis(title="",tickCount = "month", grid=False,format="%B %Y")),
    alt.Y('value',axis=alt.Axis(title="Diagnosed Cases", grid=False),stack = False),

    alt.Color('variable:N',legend=alt.Legend(title= "Contribution",offset = 10, labelFontSize = 16, titleFontSize = 20)),


).properties(
    width=800,
    height=300
).transform_filter(datum.variable == "Empirical")
cases_bar

alt.Chart(...)

In [182]:
long_sims_df["label"] = "Model (FoI=1.5) R_e_MA"

In [183]:
stream_graph_all =  alt.Chart(long_sims_df, title = "").mark_line(interpolate='monotone', opacity = 1 ,color = "#72b7b2", clip = True).encode(
    alt.X('week:T', axis=alt.Axis(title="",tickCount = "month", grid=False,format="%B %Y")),
    alt.Y('value',axis=alt.Axis(title="Estimated Rt", grid=False, labelFontSize = 16),stack = False),

    color = alt.Color('label',scale= alt.Scale(domain = ["Model (FoI=1.5) R_e_MA"], range = ["#72b7b2"]), legend=alt.Legend(title= "", offset = -190, labelFontSize = 10, titleFontSize = 10),),


).properties(
    width=800,
    height=300
).transform_filter(datum.variable == "Model (FoI=1.5) R_e_MA")

In [184]:
stream_graph_all 

alt.Chart(...)

In [167]:
case_rt = pd.read_csv("../data/case-rt-estimates_region.tsv", sep = "\t"); case_rt.head()
case_rt['label'] = "case-based Rt"

In [168]:
case_rt_plot_median = alt.Chart(case_rt, title = "").mark_line(interpolate='monotone', opacity = 0.7 ,color = "black", clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month", grid=False,format="%B %Y")),
    alt.Y('median_R',axis=alt.Axis(title="Estimated Rt", grid=False),stack = False, scale=alt.Scale(domain=(0,2.5))),
    alt.Opacity('label',legend=alt.Legend(title= "", offset = -170, labelFontSize = 10, titleFontSize = 10)),


).properties(
    width=800,
    height=300
)

case_rt_plot_bounds = alt.Chart(case_rt, title = "").mark_area(interpolate='monotone', opacity = 0.5 ,color = "black", clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month", grid=False,format="%B %Y")),
    alt.Y('R_upper_95',axis=alt.Axis(title="Estimated Rt", grid=False),stack = False, scale=alt.Scale(domain=(0, 2.5))),
    alt.Y2("R_lower_95")



).properties(
    width=800,
    height=300
)


case_rt_combined = case_rt_plot_median+ case_rt_plot_bounds

In [169]:
   (stream_plot_rt +  case_rt_combined + stream_graph_all).resolve_scale(color = "independent")

alt.LayerChart(...)

In [170]:
combined_rt_df.contribution[combined_rt_df.contribution == 'Local + Introductions'] = "Phylo Rt - Local + Introductions"

combined_rt_df.contribution[combined_rt_df.contribution == 'Local'] = "Phylo Rt - Local "

rt_line2 = alt.Chart(combined_rt_df).mark_area(interpolate='monotone', opacity = 0.4, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month", grid=False,format="%B %Y")),
    alt.Y('lower_hpd_log_50_MA',axis=alt.Axis(title="", grid=False)),
    alt.Y2('upper_hpd_log_50_MA' ),
    alt.Color("contribution", legend = alt.Legend(title = ""))
).properties(
    width=850,
    height=300
)
rt_band2 = alt.Chart(combined_rt_df).mark_area(
    opacity=0.3, interpolate='monotone'
).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False,format="%B %Y")),
    alt.Y('lower_hpd_log_95_MA', axis=alt.Axis(title="", grid=False)),# scale=alt.Scale(domain=(0.6, 1.3))),
    alt.Y2('upper_hpd_log_95_MA'), 
    alt.Color("contribution")
).properties(
    width=850,
    height=300
)

stream_plot_rt = rt_line2  + one_line

stream_plot_rt

alt.LayerChart(...)

In [171]:
combined_rt_df[(combined_rt_df.date >"2023-03-01")  & (case_rt.date <"2024-08-16")].mean_percent.mean()

1.0338071554888215

In [172]:
case_rt[(case_rt.date >"2023-03-01") & (case_rt.date <"2024-08-16")].median_R.mean()

1.0827706467917448

In [173]:
case_rt[case_rt.date >"2023-03-01"].median_R.mean()

1.0570377264948454